In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sklearn
from collections import defaultdict, Counter
from bisect import bisect_left
import matplotlib.pyplot as plt
import matplotlib
import io
import base64
import pymp
from multiprocessing import Pool, Manager, cpu_count
from IPython.display import display_html
from itertools import chain, cycle
import lightgbm as lgb
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import *
from src.util import *
from src.get_preds import *
from src import config
from src.visualize import *
from src.rewards import *
pd.set_option('display.float_format', lambda x: '%.02f' % x)

No module named 'tensorflow.python.keras.layers.preprocessing'


In [2]:
import optuna
from optuna import Trial

optuna.__version__

'2.10.0'

In [3]:
root = '../input/feedback-prize-2021'

In [4]:
df = pd.read_feather(f'{root}/train_en.fea')

In [5]:
MAX_SEQ_LEN = {}
df['len'] = df.end - df.start
max_lens = df.groupby('para_type')['len'].quantile(.999)
for i in range(len(ALL_CLASSES)):
  MAX_SEQ_LEN[id2dis[i]] = int(max_lens[i])
MAX_SEQ_LEN

{'Nothing': 223,
 'Claim': 67,
 'Evidence': 417,
 'Position': 74,
 'Concluding Statement': 245,
 'Lead': 274,
 'Counterclaim': 142,
 'Rebuttal': 142}

In [6]:
MIN_SEQ_LEN = {}
df['len'] = df.end - df.start
min_lens = df.groupby('para_type')['len'].quantile(.001)
for i in range(len(ALL_CLASSES)):
  MIN_SEQ_LEN[id2dis[i]] = int(min_lens[i])
MIN_SEQ_LEN

{'Nothing': 1,
 'Claim': 1,
 'Evidence': 6,
 'Position': 2,
 'Concluding Statement': 5,
 'Lead': 4,
 'Counterclaim': 3,
 'Rebuttal': 3}

In [7]:
MAX_SEQ_LEN = {'Nothing': 223,
 'Claim': 67,
 'Evidence': 417,
 'Position': 74,
 'Concluding Statement': 245,
 'Lead': 274,
 'Counterclaim': 142,
 'Rebuttal': 142}

In [8]:
MIN_SEQ_lens = {'Nothing': 1,
 'Claim': 1,
 'Evidence': 6,
 'Position': 2,
 'Concluding Statement': 5,
 'Lead': 4,
 'Counterclaim': 3,
 'Rebuttal': 3}

In [9]:
d = pd.read_feather(f'{root}/para_label.fea')

In [10]:
gts = {}
for row in tqdm(d.itertuples(), total=len(d)):
  gts[row.id] = decode_label_all(row.start, row.para_types)

  0%|          | 0/15594 [00:00<?, ?it/s]

In [11]:
folds = {}
for row in tqdm(df.itertuples(), total=len(df)):
  folds[row.id] = row.kfold

  0%|          | 0/175160 [00:00<?, ?it/s]

In [12]:
np.random.seed(12345)

In [13]:
gts['A8445CABFECE']

{0: [[0, 3]],
 1: [[15, 36]],
 2: [[36, 177]],
 3: [[3, 15]],
 4: [[177, 212]],
 5: [],
 6: [],
 7: []}

In [14]:
root = '../working/offline/44/'

In [15]:
# mn = 'mui.deberta-v3.rl.s1'
# mn = 'mui.electra.rl.s1'
# mn = 'mui.4'
# mn = 'large.longformer.start.len1280.seq_encoder'
# root = f'../working/offline/44/0/{mn}'

In [16]:
info = gezi.load(f'{root}/valid3.pkl')

In [17]:
df_gt = pd.read_csv(f'{root}/valid_gt.csv')

In [18]:
proba_thresh = {
    "Lead": 0.99,
    "Position": 0.85,
    "Evidence": 0.99,
    "Claim": 0.4,
    "Concluding Statement": 0.99,
    "Counterclaim": 0.5,
    "Rebuttal": 0.55,
    "Nothing": 0.5,
}
P = {
  'sep_prob': 0.5,
  'sep_prob2': 0.3,
  'sep_prob3': 0.3,
  'pre_prob': 0.5,
  'after_prob': 0.5,
  'sep_prob4': 0.55,
  'sep_prob5': 0.8,
  'link_len': 26,
}
def get_pred_bystart(x, post_adjust=True, pred_info=None):
  MIN_LEN = 2
  MIN_LEN2 = 2
  NUM_CLASSES = len(id2dis)
  pred = x['preds']
  total = len(pred)
  # by prob not logit
  probs = x['probs'] 
  # probs = x['pred']
  start_prob = x['start_probs'] if 'start_probs' in x else None
  # ic((start_prob[:,1] > start_prob[:,0]).astype(int).sum())
  pre_type = None
  # predictionstring list
  preds_list = []
  # store each pred word_id for one precitionstring
  preds = [] 
  pre_scores = np.zeros_like(probs[0])
  
  types = []
    
  pre_probs = None
  for i in range(total):    
   
    is_sep = False
    if i > 0:
      pre_cls = np.argmax(pre_scores)
      pre_prob = pre_probs[pre_cls]
      pre_type = id2dis[pre_cls]
      now_cls = pred[i]
      now_type = id2dis[now_cls]
      now_prob = probs[i][now_cls]
      
      if start_prob[i].sum() == 0:
        is_sep = True
        
      if start_prob is None:
        is_sep = pred[i] % 2 == 1
      else:
        is_sep = start_prob[i][1] >= P['sep_prob']
        
        if post_adjust and FLAGS.sep_rule:
          if i > 0:
            # 注意目前最高线上版本依然是按照第一个model取pred而不是ensemble的结果取pred信息，另外有adjacent rule待验证
            if pred[i] != pred[i - 1]:
              if start_prob[i][1] > P['sep_prob2'] and (pre_type in  ['Rebuttal', 'Counterclaim']):
                  is_sep = True
      
              # TODO this rule should be optimized...
              # if start_prob[i][1] > 0.45:
              #   # if pre_prob > proba_thresh[pre_type] and len(preds) >= min_thresh[pre_type]:
              #   if len(preds) >= min_thresh[pre_type]:
              #     is_sep = True
              
              
              if start_prob[i][1] > P['sep_prob3']:
                # if pre_prob > proba_thresh[pre_type] and len(preds) >= min_thresh[pre_type]:
                if len(preds) >= min_thresh[pre_type]:
                  is_sep = True
                                 
              if FLAGS.adjacent_rule:
                if pre_probs[pred[i - 1]] > P['pre_prob'] and probs[i][pred[i]] > P['after_prob'] and len(preds) >= min_thresh[id2dis[pred[i - 1]]]:
                  is_sep = True
            else:
              if pre_type == 'Evidence':
                if start_prob[i][1] < P['sep_prob4']:
                  is_sep = False
                  
              if len(preds) <= P['link_len'] and start_prob[i][1] < P['sep_prob5']:
                is_sep = False
                  
    if is_sep:
      if preds:  
        if pre_type != 'Nothing':
          if post_adjust:
            if len(preds) < MIN_LEN:
              # 低置信度的干脆放弃召回 更安全 pass not continue
              pass
            else:
              if pre_probs.max() > proba_thresh[pre_type]:
                preds_list.append(' '.join(preds))
                types.append(pre_type)
              # else:
              #   top_classes = np.argsort(-pre_probs,axis=0)[:2]
              #   pre_cls = top_classes[1]
              #   pre_type = id2dis[pre_cls]
              #   if pre_probs[pre_cls] > proba_thresh[pre_type]:
              #     preds_list.append(' '.join(preds))
              #     types.append(pre_type)
          else:
            preds_list.append(' '.join(preds))
            types.append(pre_type)
      
        preds = []
        pre_scores = np.zeros_like(probs[0])
              
    pre_scores += probs[i] 
    pre_probs = gezi.softmax(pre_scores)
    # pre_probs = pre_scores / len(preds)
    preds.append(str(i))
    
  if preds:
    pre_type = id2dis[np.argmax(pre_scores)]
      
    # 结尾应该更长
    if pre_type != 'Nothing':
      if post_adjust:
        if len(preds) >= MIN_LEN2:
          if pre_probs.max() > proba_thresh[pre_type]:
            preds_list.append(' '.join(preds))
            types.append(pre_type)
        # else:
        #   top_classes = np.argsort(-pre_probs,axis=0)[:2]
        #   pre_cls = top_classes[1]
        #   pre_type = id2dis[pre_cls]
        #   if pre_probs[pre_cls] > proba_thresh[pre_type]:
        #     preds_list.append(' '.join(preds))
        #     types.append(pre_type)
      else:
        preds_list.append(' '.join(preds))
        types.append(pre_type)
  return types, preds_list

In [19]:
def get_preds_(x, post_adjust=True, selected_ids=None, fold=None, folds=5):  
  # ic(post_adjust)
  pred_fn = None
  if FLAGS.token2word:
    if FLAGS.pred_method == 'end':
      pred_fn = get_pred_byend
    elif FLAGS.pred_method == 'se':
      pred_fn = get_pred_byse
    elif FLAGS.pred_method == 'start':
      pred_fn = get_pred_bystart
    else:
      raise ValueError(FLAGS.pred_method)
  else:
    if FLAGS.pred_method == 'end':
      pred_fn = get_pred_byend2
    elif FLAGS.pred_method == 'se':
      pred_fn = get_pred_byse2
    elif FLAGS.pred_method == 'start':
      pred_fn = get_pred_bystart2
    else:
      raise ValueError(FLAGS.pred_method)
  # ic(pred_fn)

  total = len(x['id'])
  # with gezi.Timer('get_preds'):
  # ic(FLAGS.openmp)
  ids_list, types_list, preds_list = [], [], []
  for i in tqdm(range(total), desc='get_preds', leave=False):
    id = x['id'][i]
    if selected_ids is not None and id not in selected_ids:
      continue
    if fold is not None:
      if i % folds != fold:
        continue
    x_ = {}
    for key in x: 
      x_[key] = x[key][i]
    types, preds = pred_fn(x_, post_adjust=post_adjust)
    ids_list.extend([id] * len(types))
    types_list.extend(types)
    preds_list.extend(preds)
 
  m = {
    'id': ids_list,
    'class': types_list,
    'predictionstring': preds_list
  }

  df = pd.DataFrame(m)
    
  return df

def get_preds(x, post_adjust=True, selected_ids=None, folds=5):  
  if selected_ids is not None:
    return get_preds_(x, post_adjust, selected_ids)
  else:
    try:
      dfs = Manager().dict()
      with pymp.Parallel(folds) as p:
        for i in p.range(folds):
          dfs[i] = get_preds_(x, post_adjust, fold=i, folds=folds)
      return pd.concat(dfs.values())
    except Exception:
      return get_preds_(x, post_adjust)

In [20]:
gezi.init_flags()

In [21]:
P = {
  'sep_prob': 0.5,
  'sep_prob2': 0.3,
  'sep_prob3': 0.3,
  'pre_prob': 0.5,
  'after_prob': 0.5,
  'sep_prob4': 0.55,
  'sep_prob5': 0.8,
  'link_len': 26,
}

def objective(trial):
    
    # params = {
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',
    #     'boosting_type': 'gbdt',
    #     'boost_from_average': True,
    #     'num_threads': 4,
    #     'random_state': 42,
        
    #     'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
    #     'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 200),sc
    #     'min_child_weight': trial.suggest_loguniform('min_child_weight', 0.001, 0.1),
    #     'max_depth': trial.suggest_int('max_depth', 1, 100),
    #     'bagging_fraction': trial.suggest_loguniform('bagging_fraction', .5, .99),
    #     'feature_fraction': trial.suggest_loguniform('feature_fraction', .5, .99),
    #     'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.1, 2),
    #     'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.1, 2)

    # return np.mean(scores)

   
    # proba_thresh['Lead'] = trial.suggest_uniform('Lead', 0.9, 0.999)
    for key in P:
      if 'prob' in key:
        P[key] = trial.suggest_uniform(key, 0., 1.)
      else:
        if key == 'link_len':
          P[key] = trial.suggest_int(key, 8, 40)
    df_pred = get_preds(info, post_adjust=True, folds=50)
    res = calc_metrics(df_gt, df_pred)
    score = res['f1/Overall']
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2022-02-13 11:20:31,577] A new study created in memory with name: no-name-7126c0d2-3553-44f5-a8b3-8d54815e88db


get_preds:   0%|          | 0/15594 [00:00<?, ?it/s]

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

[I 2022-02-13 11:21:11,054] Trial 0 finished with value: 0.6983739822483165 and parameters: {'sep_prob': 0.9788090481303899, 'sep_prob2': 0.40640243412394195, 'sep_prob3': 0.26393042971569003, 'pre_prob': 0.374689644420091, 'after_prob': 0.6287910908130103, 'sep_prob4': 0.5549932599400673, 'sep_prob5': 0.621512204326915, 'link_len': 13}. Best is trial 0 with value: 0.6983739822483165.


get_preds:   0%|          | 0/15594 [00:00<?, ?it/s]

...

[I 2022-02-13 11:36:53,863] Trial 46 finished with value: 0.707608865399083 and parameters: {'sep_prob': 0.8704968962695425, 'sep_prob2': 0.2907209264580516, 'sep_prob3': 0.18236773938498066, 'pre_prob': 0.9995905323229793, 'after_prob': 0.5437144948975896, 'sep_prob4': 0.5314187003355211, 'sep_prob5': 0.6635620883892975, 'link_len': 19}. Best is trial 33 with value: 0.7093606481664846.


get_preds:   0%|          | 0/15594 [00:00<?, ?it/s]

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

[I 2022-02-13 11:37:17,376] Trial 47 finished with value: 0.7022581872972211 and parameters: {'sep_prob': 0.6959387854391654, 'sep_prob2': 0.25225247296944636, 'sep_prob3': 0.33029010540959475, 'pre_prob': 0.3731860858857883, 'after_prob': 0.31305341972985495, 'sep_prob4': 0.32043166750780516, 'sep_prob5': 0.25305149468352794, 'link_len': 24}. Best is trial 33 with value: 0.7093606481664846.


get_preds:   0%|          | 0/15594 [00:00<?, ?it/s]

In [ ]:
study.best_params

{'sep_prob': 0.7261767844346143,
 'sep_prob2': 0.9824113360256063,
 'sep_prob3': 0.44774550783732686,
 'pre_prob': 0.9273050195499939,
 'after_prob': 0.17105861734159292,
 'sep_prob4': 0.43525772456940753,
 'sep_prob5': 0.13874548968602074,
 'link_len': 38}

In [ ]:
gezi.init_flags()

In [ ]:
df_pred = get_preds(info, post_adjust=True, folds=50)

get_preds:   0%|          | 0/15594 [00:00<?, ?it/s]

In [ ]:
res = calc_metrics(df_gt, df_pred)
res

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

{'f1/Overall': 0.7068093341845719,
 'f1/Claim': 0.6635564740017117,
 'acc/Claim': 0.6970507619778533,
 'recall/Claim': 0.6331335019617996,
 'f1/Evidence': 0.7698335185621752,
 'acc/Evidence': 0.7904832165252674,
 'recall/Evidence': 0.7502352143185349,
 'f1/Position': 0.7333266000067333,
 'acc/Position': 0.7625148778267871,
 'recall/Position': 0.7062905317769131,
 'f1/Concluding Statement': 0.8712900274473925,
 'acc/Concluding Statement': 0.8614226789203271,
 'recall/Concluding Statement': 0.8813860506441582,
 'f1/Lead': 0.8354062348570506,
 'acc/Lead': 0.8371641307866623,
 'recall/Lead': 0.8336557059961315,
 'f1/Counterclaim': 0.5798785318193821,
 'acc/Counterclaim': 0.5942630344578748,
 'recall/Counterclaim': 0.5661739429357168,
 'f1/Rebuttal': 0.49437395259755806,
 'acc/Rebuttal': 0.5141932270916335,
 'recall/Rebuttal': 0.4760258183494698}

In [ ]:
# res = calc_metrics(df_gt, link_evidence(df_pred))
# res